In [1]:
#!pip install neptune
#!pip install optuna
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from medmnist import DermaMNIST
import random
import neptune
import itertools
import optuna

In [2]:
torch.cuda.empty_cache()
class CNN(nn.Module):
    def __init__(self, dropout_rate):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn4 = nn.BatchNorm2d(256)
        self.conv5 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn5 = nn.BatchNorm2d(512)

        self.pool = nn.MaxPool2d(kernel_size=2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

        # Oblicz rozmiar spłaszczonego tensora
        self.flattened_size = self.get_flattened_size((3, 64, 64))
        self.fc1 = nn.Linear(self.flattened_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 7)  # 7 klas w DermaMNIST

    def get_flattened_size(self, input_shape):
        x = torch.zeros(1, *input_shape)
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = self.pool(self.relu(self.conv4(x)))
        x = self.pool(self.relu(self.conv5(x)))
        return x.view(1, -1).size(1)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.pool(x)
        x = self.relu(self.bn4(self.conv4(x)))
        x = self.pool(x)
        x = self.relu(self.bn5(self.conv5(x)))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Funkcja celu dla Optuny
def objective(trial):
    # Hiperparametry do tuningu
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128, 256])
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-2)
    dropout_rate = trial.suggest_uniform("dropout_rate", 0.2, 0.5)

    # Dane
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(10),
        transforms.RandomAutocontrast(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.Normalize((0.5,), (0.5,))
    ])
    train_dataset = DermaMNIST(root='./data', split="train", download=True, transform=transform, size=64)
    test_dataset = DermaMNIST(root='./data', split="test", download=True, transform=transform, size=64)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Model
    model = CNN(dropout_rate).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

    # Trening
    for epoch in range(10):  # Mała liczba epok do szybkiego tuningu
        model.train()
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            target = target.long()  # Ensure target is of type torch.long
            if len(target.size()) > 1:
                target = target.squeeze()  # Remove extra dimensions
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()

    # Ewaluacja
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            target = target.long()  # Ensure target is of type torch.long
            if len(target.size()) > 1:
                target = target.squeeze()  # Remove extra dimensions
            outputs = model(data)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == target).sum().item()
            total += target.size(0)

    accuracy = correct / total
    return 1 - accuracy  # Minimalizujemy 1 - accuracy

# Główna funkcja
if __name__ == "__main__":
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    study = optuna.create_study(direction="minimize")  # Minimalizacja (1 - accuracy)
    study.optimize(objective, n_trials=50)

    # Najlepsze hiperparametry
    print("Najlepsze hiperparametry: ", study.best_params)
    print("Najlepszy wynik: ", 1 - study.best_value)

[I 2025-01-24 16:22:34,291] A new study created in memory with name: no-name-54c798b0-5ec0-4862-ae66-c3bba9698b24
C:\Users\Kamil\AppData\Local\Temp\ipykernel_34100\2442891932.py:56: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-2)
C:\Users\Kamil\AppData\Local\Temp\ipykernel_34100\2442891932.py:57: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform("dropout_rate", 0.2, 0.5)


Using downloaded and verified file: ./data\dermamnist_64.npz
Using downloaded and verified file: ./data\dermamnist_64.npz


[I 2025-01-24 16:23:41,679] Trial 0 finished with value: 0.24039900249376556 and parameters: {'batch_size': 64, 'learning_rate': 0.0003359255610995169, 'dropout_rate': 0.27239074012592335}. Best is trial 0 with value: 0.24039900249376556.


Using downloaded and verified file: ./data\dermamnist_64.npz
Using downloaded and verified file: ./data\dermamnist_64.npz


[I 2025-01-24 16:24:49,103] Trial 1 finished with value: 0.23840399002493762 and parameters: {'batch_size': 64, 'learning_rate': 0.0008351745388035985, 'dropout_rate': 0.39569484252809206}. Best is trial 1 with value: 0.23840399002493762.


...

Using downloaded and verified file: ./data\dermamnist_64.npz
Using downloaded and verified file: ./data\dermamnist_64.npz


[I 2025-01-24 17:22:15,012] Trial 48 finished with value: 0.24139650872817953 and parameters: {'batch_size': 256, 'learning_rate': 0.000791091953195575, 'dropout_rate': 0.41351546938072986}. Best is trial 31 with value: 0.22892768079800496.


Using downloaded and verified file: ./data\dermamnist_64.npz
Using downloaded and verified file: ./data\dermamnist_64.npz


[I 2025-01-24 17:23:23,540] Trial 49 finished with value: 0.2483790523690773 and parameters: {'batch_size': 128, 'learning_rate': 0.0004531328278595781, 'dropout_rate': 0.4398564016706555}. Best is trial 31 with value: 0.22892768079800496.


Najlepsze hiperparametry:  {'batch_size': 32, 'learning_rate': 0.00018826581273916093, 'dropout_rate': 0.4906930122104726}
Najlepszy wynik:  0.771072319201995
